# Import libraries

In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from dataset import NCFDataset
from recom_ncf import NCFRecommender
from evaluation import Evaluation
from helpers.index_manager import IndexManager
from helpers.splitter import Splitter
from helpers.dataloader_custom_functions import collate_fn, worker_init_fn
from helpers.image_cacher import CacheType

# Loading data

In [2]:
df = pd.read_csv('../data/interaction-clean.csv')[['user_id', 'item_id', 'rating_imp', 'timestamp']]
df.head()

,user_id,item_id,rating_imp,timestamp
0,76561197960432447,10,1,1738278781
1,76561198071230926,10,1,1736206418
2,76561198206216352,10,1,1738041574
3,76561198110801124,10,1,1738015332
4,76561199813732773,10,1,1737853720


In [3]:
df_metadata = pd.read_csv('../data/metadata-features.csv')[['item_id']]
df_metadata.head()

,item_id
0,1430720
1,1430740
2,1430100
3,1430680
4,1430970


# Indexing data

In [4]:
index_manager = IndexManager()
index_manager.fit(df_interaction=df)
index_manager.transform_interactions(df, inplace=True)
index_manager.transform_metadata(df_metadata, inplace=True)

Indexed 836887 users and 69001 items
User index range: 0-836886
Item index range: 0-69000


""
item_id
89949
36882
36872
36881
36886
...
41579
41580
41581


# Creating datasets

In [5]:
from helpers.image_dataloader import ImageDataLoader
image_dataloader = ImageDataLoader(file_dir='D:/image-features', index_manager=index_manager, cache_type=CacheType.UNLIMITED)

In [6]:
splitter = Splitter(df)
df_train, df_val, df_test = splitter.leave_k_out_split()

Splitting data with leave-2-out strategy (1 for validation, 1 for testing)
Total users: 836887
Interactions per user: min=1, max=1035, avg=1.4
Note: 706515 users have fewer than 2 interactions.
These users will be placed entirely in the training set.
Split complete: 1156226 total interactions
Train set: 895482 interactions (77.4%)
Validation set: 130372 interactions (11.3%)
Test set: 130372 interactions (11.3%)


In [7]:
train_dataset = NCFDataset(df_train, image_dataloader=image_dataloader)
val_dataset = NCFDataset(df_val, image_dataloader=image_dataloader)

dataloader_params = {
    'batch_size': 2**16,
    'num_workers': 4,
    'persistent_workers': True,
    'prefetch_factor': 2,
    'pin_memory': True,
    'pin_memory_device': 'cuda',
    'collate_fn': collate_fn,
    'worker_init_fn': worker_init_fn,
}

train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloader_params)
eval_dataloader = DataLoader(val_dataset, shuffle=False, **dataloader_params)

# Training model

In [8]:
params = {'factors': 8, 'mlp_user_item_dim': 128, 'learning_rate': 0.005, 'epochs': 5, 'optimizer': 'adagrad', 'dropout': 0.0, 'weight_decay': 0.0001, 'loss_fn': 'mse', 'image_dim': 128}

unique_users = index_manager.get_indexed_users()
unique_items = index_manager.get_indexed_items()

model = NCFRecommender(unique_users, unique_items, image_dataloader=image_dataloader, **params)
model.fit(train_dataloader, eval_dataloader)

Epoch 1/5
Train loss: 0.227885, Validation loss: 0.172998
Epoch 2/5
Train loss: 0.142032, Validation loss: 0.147498
Epoch 3/5
Train loss: 0.131513, Validation loss: 0.144505
Epoch 4/5
Train loss: 0.126492, Validation loss: 0.142005
Epoch 5/5
Train loss: 0.120630, Validation loss: 0.140023
Training completed!


# Evaluating model

In [9]:
evaluator = Evaluation(recommender=model, test_data=df_test)
metrics = evaluator.evaluate()

Creating ground truth sets...
Generating predictions...
Processing 1 of 130372 users... (0.00%)
Memory usage: 0.378173828125 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 128 to 141
Increased item batch size from 1024 to 1126
Processing 129 of 130372 users... (0.10%)
Memory usage: 0.226806640625 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 141 to 155
Increased item batch size from 1126 to 1239
Processing 270 of 130372 users... (0.21%)
Memory usage: 0.2470703125 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 155 to 170
Increased item batch size from 1239 to 1363
Processing 425 of 130372 users... (0.33%)
Memory usage: 0.274169921875 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 170 to 187
Increased item batch size from 1363 to 1499
Processing 595 of 130372 users... (0.46%)
Memory usage: 0.3037109375 . Increasing batch size with increasing rat

In [10]:
metrics

{'Hit Ratio@10': 0.03713220630196668,
 'NDCG@10': 0.017960454006970984,
 'Recall@10': 0.03713220630196668}